In [97]:
import pickle
import itertools
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import ranksums
from scipy import stats

In [34]:
pickle_in = open("errors.p","rb")
errors = pickle.load(pickle_in)


In [120]:
for k in errors.keys():
    print(k)

Lasso
LassoLars
LogisticRegression
PassiveAggressiveClassifier
RidgeRegression


In [129]:
stats.friedmanchisquare(*[errors[k] for k in errors.keys()])

FriedmanchisquareResult(statistic=77.6822742474915, pvalue=5.393121464539751e-16)

In [125]:
a = (errors['Lasso'], errors['LassoLars'], errors['RidgeRegression'])

In [127]:
a

(array([4.13882288e-01, 1.15363015e-01, 5.42753071e-02, 1.58222633e-01,
        3.04054530e-01, 1.37659172e+01, 1.19534275e+01, 2.88284680e-01,
        2.90387118e-01, 1.02651881e-01, 1.87238521e-01, 1.97775388e-01,
        5.28829699e-02, 9.22663094e-02, 1.99527809e-01, 6.23552082e-01,
        3.32695957e-01, 5.30382940e+00, 2.05966947e-01, 1.93908792e+01,
        1.50188615e-01, 2.29177702e-01, 2.11865724e-01, 4.99855262e+00,
        1.85841190e-01, 7.45048470e-01, 1.22514767e-01, 2.27714770e-01,
        4.02647102e-01, 1.64855940e-01, 5.43146603e-01, 1.52682009e-01,
        1.65994439e-01, 1.57273335e-01, 2.39626883e+00, 9.69027494e-01,
        1.89212304e-01, 4.78189576e-01, 6.52477664e-01, 1.47912354e-01,
        9.94503637e-01, 1.27307256e+00, 1.30103609e-01, 2.50498216e-01,
        1.72587122e-01, 2.05995953e-01, 1.83365320e+00, 1.34855726e-01,
        5.37540499e-02, 6.10535750e+00, 2.62207860e-01, 4.11415101e+01,
        1.68365953e+01, 8.61114779e-01, 5.31342042e-01, 3.071452